# Building Secure Chatbots with Amazon Bedrock Guardrails and Strands SDK

## Overview

In this comprehensive notebook, you'll learn how to build secure, production-ready chatbots by combining Amazon Bedrock Guardrails with the Strands SDK. We'll cover:

1. **Amazon Bedrock Guardrails**: Creating and managing guardrails to protect your AI applications
2. **Strands Agent Integration**: Seamlessly integrating guardrails with Strands agents
3. **Conversation Management**: Leveraging Strands' built-in session management for chatbot applications

By the end of this notebook, you'll understand how to create chatbots that are not only conversational but also safe, compliant, and appropriate for production use cases.

### Why Secure Chatbots Matter

Modern chatbots handle sensitive customer interactions and must:
- Prevent harmful or inappropriate content
- Protect personally identifiable information (PII)
- Comply with industry regulations
- Maintain conversation context securely
- Provide consistent safety standards across all interactions

This notebook demonstrates how to achieve all of these goals using AWS services and the Strands SDK.

## Notebook Details

|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|AWS Services used   |Amazon Bedrock, Amazon Bedrock Guardrails          |
|SDK used            |Strands Agents SDK                                 |
|Key Concepts        |Guardrails, Content Filtering, PII Protection, Conversation Management|
|Agent Structure     |Secure chatbot with guardrails and session management|

## Key Features

* **Comprehensive Guardrail Configuration**: Learn how to configure all guardrail components including content filters, denied topics, word filters, PII protection, and contextual grounding
* **Version Management**: Understand DRAFT vs VERSIONED guardrails and how to iterate safely
* **Seamless Strands Integration**: Add guardrails to your Strands agents with simple configuration
* **Input/Output Redaction**: Control how blocked content appears in conversation history
* **Session Management**: Understand conversation persistence options for production deployments
* **Production-Ready Examples**: See complete, working examples ready for deployment

### Import Dependencies

In [ ]:
import boto3
import json
import sys
import pprint
sys.path.append('../')
from util.tagging import standard_tags_kv_lc
from strands import Agent
from strands.models import BedrockModel

print(f"Boto3 version: {boto3.__version__}")
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
# Initialize AWS clients
boto3_session = boto3.session.Session()
region = boto3_session.region_name

bedrock_client = boto3.client('bedrock')
bedrock_runtime = boto3.client('bedrock-runtime')

print(f"AWS Region: {region}")

---

# Part 1: Amazon Bedrock Guardrails

Amazon Bedrock Guardrails provides safeguards that you can configure for your generative AI applications based on your use cases and responsible AI policies. You can create multiple guardrails tailored to different use cases and apply them across multiple foundation models (FMs), providing a consistent user experience and standardizing safety and privacy controls across generative AI applications. You can use guardrails for both model prompts and responses with natural language.

## Key Benefits

- **Consistent Safety**: Apply the same safety standards across multiple foundation models
- **Customizable Protection**: Tailor guardrails to your specific use case and industry requirements
- **Real-time Filtering**: Filter both input prompts and model responses in real-time
- **Compliance Support**: Help meet regulatory and organizational compliance requirements

## Core Components

1. **Content Filters** - Block harmful content (hate, violence, sexual content, etc.)
2. **Denied Topics** - Prevent discussions on specific topics
3. **Word Filters** - Block specific words or phrases
4. **Sensitive Information Filters** - Detect and handle PII and sensitive data
5. **Contextual Grounding Checks** - Prevent hallucinations and ensure factual accuracy
6. **Automated Reasoning Checks** - Validate responses against logical rules and policies

## Creating a Guardrail

Guardrails for Amazon Bedrock have multiple components which include Content Filters, Denied Topics, Word and Phrase Filters, and Sensitive Word (PII & Regex) Filters. For a full list check out the policies to configure when you [create your guardrail](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-components.html).

### Understanding Guardrail Configuration

In this example, we're creating a guardrail specifically designed for a financial services chatbot that needs to prevent the AI from providing fiduciary advice. This demonstrates a real-world use case where regulatory compliance is critical.

**Configuration Components Explained:**

1. **Topic Policy (Denied Topics)**:
   - Defines specific topics that should be blocked
   - Uses natural language definitions and examples
   - In our case: preventing fiduciary investment advice

2. **Content Policy (Content Filters)**:
   - Filters harmful content categories with configurable strength levels
   - Strength levels: NONE, LOW, MEDIUM, HIGH
   - Separate settings for input (user prompts) and output (model responses)

3. **Word Policy (Word Filters)**:
   - Blocks specific words/phrases (exact match)
   - Includes managed word lists (like profanity)
   - Custom words related to financial advice in our example

4. **Sensitive Information Policy**:
   - Handles PII detection and protection
   - Actions: BLOCK (reject) or ANONYMIZE (mask)
   - Supports custom regex patterns for domain-specific data

5. **Contextual Grounding Policy**:
   - Prevents hallucinations in RAG applications
   - GROUNDING: Ensures responses are based on source material
   - RELEVANCE: Ensures responses are relevant to the query

In [ ]:
create_response = bedrock_client.create_guardrail(
    name='secure-chatbot-guardrail',
    description='Prevents the chatbot from providing fiduciary advice and filters harmful content.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
    },
    blockedInputMessaging="""I can provide general information about our products and services, but I can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details.""",
    blockedOutputsMessaging="""I can provide general information about our products and services, but I can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details.""",
    tags=[
        {'key': 'purpose', 'value': 'secure-chatbot'},
        {'key': 'environment', 'value': 'workshop'}
    ] + standard_tags_kv_lc
)

pp.pprint(create_response)

### Analyzing the Guardrail Creation Response

The `create_guardrail` API call returns several important pieces of information:

- **guardrailId**: Unique identifier for this guardrail
- **guardrailArn**: Amazon Resource Name for the guardrail
- **version**: Initially created as 'DRAFT' - allows for testing and iteration
- **createdAt**: Timestamp of creation

In [ ]:
# Store the guardrail identifiers for later use
guardrail_id = create_response['guardrailId']
guardrail_arn = create_response['guardrailArn']

print(f"Guardrail ID: {guardrail_id}")
print(f"Guardrail ARN: {guardrail_arn}")

## Guardrail Version Management

Amazon Bedrock Guardrails supports version management to enable safe iteration and deployment:

**DRAFT vs VERSIONED Guardrails:**
- **DRAFT**: Mutable version for development and testing
- **VERSIONED**: Immutable snapshots for production use
- You can have one DRAFT and multiple numbered versions (1, 2, 3, etc.)

**Best Practices:**
1. Develop and test with DRAFT versions
2. Create numbered versions for production deployment
3. Use version numbers to track changes and enable rollbacks
4. Tag versions with meaningful descriptions

In [ ]:
# Create a version for production use
version_response = bedrock_client.create_guardrail_version(
    guardrailIdentifier=guardrail_id,
    description='Initial production version of secure chatbot guardrail'
)

guardrail_version = version_response['version']
print(f"Created guardrail version: {guardrail_version}")

In [ ]:
# List all versions of our guardrail
list_response = bedrock_client.list_guardrails(
    guardrailIdentifier=guardrail_arn,
    maxResults=5
)

pp.pprint(list_response)

## Understanding Guardrail Updates

Guardrail updates demonstrate the iterative nature of responsible AI development:

**Update Process:**
1. Only DRAFT versions can be updated
2. Updates modify the entire guardrail configuration
3. Changes are immediately available for testing
4. Create new versions to preserve update history

**Content Filter Strength Levels:**
- **NONE**: No filtering
- **LOW**: Minimal filtering, only obvious violations
- **MEDIUM**: Balanced filtering
- **HIGH**: Strict filtering, may catch borderline cases

For this workshop, let's demonstrate the update process by changing the HATE filter strength from HIGH to MEDIUM.

In [ ]:
# Update the DRAFT version - changing HATE filter from HIGH to MEDIUM
update_response = bedrock_client.update_guardrail(
    guardrailIdentifier=guardrail_id,
    name='secure-chatbot-guardrail',
    description='Guardrail for secure chatbot with updated HATE filter',
    topicPolicyConfig={'topicsConfig': [{
        'name': 'Fiduciary Advice',
        'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
        'examples': [
            'What stocks should I invest in for my retirement?',
            'Is it a good idea to put my money in a mutual fund?',
            'How should I allocate my 401(k) investments?',
            'What type of trust fund should I set up for my children?',
            'Should I hire a financial advisor to manage my investments?'
        ],
        'type': 'DENY'
    }]},
    contentPolicyConfig={'filtersConfig': [
        {'type': 'SEXUAL', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
        {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
        {'type': 'HATE', 'inputStrength': 'MEDIUM', 'outputStrength': 'MEDIUM'},  # Changed!
        {'type': 'INSULTS', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
        {'type': 'MISCONDUCT', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
        {'type': 'PROMPT_ATTACK', 'inputStrength': 'HIGH', 'outputStrength': 'NONE'}
    ]},
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'}, {'text': 'investment recommendations'},
            {'text': 'stock picks'}, {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'}, {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'}, {'text': 'trust fund setup'},
            {'text': 'investment strategy'}, {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [{'type': 'PROFANITY'}]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'}, {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'}, {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'}, {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [{
            'name': 'Account Number',
            'description': 'Matches account numbers in the format XXXXXX1234',
            'pattern': r'\b\d{6}\d{4}\b',
            'action': 'ANONYMIZE'
        }]
    },
    contextualGroundingPolicyConfig={'filtersConfig': [
        {'type': 'GROUNDING', 'threshold': 0.75},
        {'type': 'RELEVANCE', 'threshold': 0.75}
    ]},
    blockedInputMessaging="""I can provide general information about our products and services, but I can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details.""",
    blockedOutputsMessaging="""I can provide general information about our products and services, but I can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details."""
)

print("✅ Updated DRAFT guardrail - changed HATE filter to MEDIUM")

In [ ]:
# Get the updated DRAFT version to verify changes
get_response = bedrock_client.get_guardrail(
    guardrailIdentifier=guardrail_id,
    guardrailVersion='DRAFT'
)

print("\nUpdated DRAFT Guardrail Configuration:")
pp.pprint(get_response)

In [ ]:
# Create version 2 from our updates
version_response_2 = bedrock_client.create_guardrail_version(
    guardrailIdentifier=guardrail_id,
    description='Version 2: Updated HATE filter to MEDIUM strength'
)

print(f"Created Version 2: {version_response_2['version']}")
print(f"Description: {version_response_2.get('description', 'N/A')}")

In [ ]:
# List all versions to see our version history
all_versions = bedrock_client.list_guardrails(
    guardrailIdentifier=guardrail_arn,
    maxResults=10
)

print("\nAll Guardrail Versions:")
pp.pprint(all_versions)

## Testing the Guardrail Directly

Before integrating with our chatbot, let's test the guardrail directly using the `apply_guardrail` API. This helps verify that our guardrail configuration works as expected.

In [ ]:
def test_guardrail(text, source_type='INPUT'):
    """
    Test function to check if input/output is blocked by guardrail.
    
    Args:
        text: The text to test
        source_type: 'INPUT' or 'OUTPUT'
    """
    response = bedrock_runtime.apply_guardrail(
        guardrailIdentifier=guardrail_id,
        guardrailVersion=str(guardrail_version),
        source=source_type,
        content=[{"text": {"text": text}}]
    )

    print(f"\n{'='*60}")
    print(f"Testing {source_type}: {text[:80]}...")
    print(f"{'='*60}")
    print(f"Action: {response.get('action')}")
    
    action = response.get('action')
    is_blocked = action == 'GUARDRAIL_INTERVENED'
    
    if is_blocked:
        print(f"\n⚠️  Content BLOCKED")
        print(f"Reason: {response.get('actionReason', 'Not specified')}")
        
        # Check for blocked topics
        assessments = response.get('assessments', [])
        if assessments and 'topicPolicy' in assessments[0]:
            blocked_topics = [topic.get('name') for topic in
                            assessments[0]['topicPolicy'].get('topics', [])
                            if topic.get('action') == 'BLOCKED']
            if blocked_topics:
                print(f"Blocked topics: {', '.join(blocked_topics)}")
        
        # Show blocked message
        if 'outputs' in response and response['outputs']:
            print(f"\nBlocked message: {response['outputs'][0].get('text', 'None')}")
    
    else:
        print(f"\n✅ Content ALLOWED (action: NONE)")
        
        # Check for PII anonymization
        assessments = response.get('assessments', [])
        if assessments and 'sensitiveInformationPolicy' in assessments[0]:
            pii_policy = assessments[0]['sensitiveInformationPolicy']
            
            # Check PII entities
            pii_entities = pii_policy.get('piiEntities', [])
            anonymized = [pii for pii in pii_entities if pii.get('action') == 'ANONYMIZED']
            blocked_pii = [pii for pii in pii_entities if pii.get('action') == 'BLOCKED']
            
            if anonymized:
                print(f"\n🔒 PII Anonymized:")
                for pii in anonymized:
                    print(f"   - {pii.get('type')}: '{pii.get('match')}' → {{{pii.get('type')}}}")
            
            if blocked_pii:
                print(f"\n🚫 PII Blocked:")
                for pii in blocked_pii:
                    print(f"   - {pii.get('type')}: '{pii.get('match')}'")
        
        # Show modified output if PII was anonymized
        if 'outputs' in response and response['outputs']:
            modified_text = response['outputs'][0].get('text', '')
            if modified_text:
                print(f"\n📝 Modified output: {modified_text}")

    return response

In [ ]:
# Test with safe input
test_guardrail("Tell me about general financial literacy concepts.")

In [ ]:
# Test with input that should be blocked
test_guardrail("What stocks should I invest in for my retirement?")

In [ ]:
# Test with PII that should be anonymized
# Note: When PII is set to ANONYMIZE (not BLOCK), the action will be \"NONE\"
# This means the request is allowed to proceed, but with PII masked in the output
# The guardrail only returns action=\"GUARDRAIL_INTERVENED\" when content is BLOCKED
test_guardrail("My email is john.doe@example.com and my phone is 555-1234")

In [ ]:
# Test with PII that should be BLOCKED (SSN)
# SSN is configured with action=BLOCK, so this should return action=\"GUARDRAIL_INTERVENED\"
test_guardrail("My social security number is 123-45-6789")

---

# Part 2: Integrating Guardrails with Strands Agents

Now that we've created and tested our guardrail, let's integrate it with a Strands Agent. The Strands SDK makes it incredibly easy to add guardrails to your agents through the BedrockModel configuration.

## Why Integrate Guardrails with Agents?

When building chatbots and conversational AI applications:
- **Automatic Protection**: Every interaction is automatically filtered
- **Consistent Safety**: Same guardrails apply across all agent capabilities
- **Tool Safety**: Guardrails protect both direct responses and tool-augmented responses
- **Conversation Context**: Guardrails work seamlessly with conversation history

## Creating a Guardrail-Protected Bedrock Model

The integration is straightforward - simply pass the guardrail configuration to the BedrockModel:

**Key Parameters:**
- `guardrail_id`: The ID of your guardrail
- `guardrail_version`: Version number or "DRAFT"
- `guardrail_trace`: Enable "enabled" for debugging and monitoring

In [ ]:
# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    guardrail_id=guardrail_id,
    guardrail_version=str(guardrail_version),
    guardrail_trace="enabled",  # Enable trace for debugging
    temperature=0.7,
    region_name=region
)

print("✅ Created guardrail-protected Bedrock model")

## Creating a Secure Chatbot Agent

Now let's create a chatbot agent that uses our guardrail-protected model. This agent will automatically benefit from all the safety features we configured.

In [ ]:
# Create a secure chatbot agent
secure_chatbot = Agent(
    system_prompt="""You are a helpful customer service assistant for a financial services company. 
    You can answer general questions about products and services, but you cannot provide 
    personalized financial advice. Always maintain a professional and helpful tone.""",
    model=bedrock_model
)

print("✅ Created secure chatbot agent with guardrails")

## Testing the Secure Chatbot

Let's test our chatbot with various inputs to see how the guardrails protect it.

In [ ]:
def test_chatbot(prompt):
    """
    Test the chatbot and check for guardrail interventions.
    """
    print(f"\n{'='*60}")
    print(f"User: {prompt}")
    print(f"{'='*60}")
    
    try:
        response = secure_chatbot(prompt)
        
        # Check if guardrail intervened
        if hasattr(response, 'stop_reason') and response.stop_reason == "guardrail_intervened":
            print("\n⚠️  GUARDRAIL INTERVENED!")
            print("The request was blocked by the guardrail.")
        else:
            print(f"\nAssistant: {response}")
            
        return response
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        return None

In [ ]:
# Test 1: Safe general question
test_chatbot("What types of savings accounts do you offer?")

In [ ]:
# Test 2: Question that should be blocked (fiduciary advice)
test_chatbot("What stocks should I invest in for my retirement?")

In [ ]:
# Test 3: Question with SSN (which we have a blocking action for) and PII (should anonymize), Blocking actions have precedence, so the full question is blocked.
test_chatbot("I need help with my account. My SSN is 123-45-6789 and my email is john@example.com")

In [ ]:
# Test 4: Follow-up question to verify conversation context
test_chatbot("Can you tell me more about the interest rates?")

## Inspecting Conversation History

Let's examine how guardrails affect the conversation history:

In [ ]:
# Display conversation history
print("\nConversation History:")
print(json.dumps(secure_chatbot.messages, indent=2))

## Advanced: Configuring Input/Output Redaction

For chatbot applications, you may want to control how blocked content appears in conversation history. Strands provides configuration options for this:

**Redaction Options:**
- `guardrail_redact_input`: Automatically redact blocked user inputs in history
- `guardrail_redact_output`: Automatically redact blocked model outputs in history
- `guardrail_redact_input_message`: Custom message to replace blocked inputs
- `guardrail_redact_output_message`: Custom message to replace blocked outputs

**Why This Matters:**
- Prevents follow-up questions from being blocked by the same content
- Maintains clean conversation history
- Provides clear feedback to users about what was blocked

In [ ]:
# Create a model with redaction enabled
redacting_model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    guardrail_id=guardrail_id,
    guardrail_version=str(guardrail_version),
    guardrail_trace="enabled",
    guardrail_redact_input=True,
    guardrail_redact_output=True,
    guardrail_redact_input_message="[Content blocked by guardrail - please rephrase your question]",
    guardrail_redact_output_message="[Response blocked by guardrail]",
    temperature=0.7,
    region_name=region
)

# Create agent with redaction
redacting_chatbot = Agent(
    system_prompt="""You are a helpful customer service assistant for a financial services company. 
    You can answer general questions about products and services, but you cannot provide 
    personalized financial advice. Always maintain a professional and helpful tone.""",
    model=redacting_model
)

print("✅ Created chatbot with input/output redaction enabled")

In [ ]:
# Test with safe question first
redacting_chatbot("What are your business hours?")

In [ ]:
# Test with blocked content
redacting_chatbot("How should I allocate my 401(k) investments?")

In [ ]:
# Show how the blocked content appears in history
print("\nConversation History with Redaction:")
print(json.dumps(redacting_chatbot.messages, indent=2))

## Understanding Conversation Management in Strands

One of the key features that makes Strands ideal for building chatbots is its built-in conversation management. The Strands Agent automatically maintains conversation history in memory during runtime, providing context across multiple interactions within a single session.

### Built-in Memory (Default)

For quick prototyping and development, Strands provides automatic in-memory conversation management without any configuration. This is what we've been using in all our examples above - the agent automatically remembers the conversation context.

### Session Persistence for Production

For production deployments, you'll want to persist conversations across application restarts or distribute them across multiple servers. Strands provides several built-in options:

**Note:** The `redacting_chatbot` combines:
- ✅ Guardrails for safety and compliance
- ✅ Input/output redaction for clean conversation history
- ✅ Automatic conversation memory
- ✅ Professional system prompt

The only additional step for production would be to add a session manager (FileSessionManager, S3SessionManager, or custom) based on your deployment architecture.

#### 1. In-Memory (Default)
- **Use Case**: Development, testing, single-session applications
- **Pros**: Fast, no configuration needed
- **Cons**: Lost on application restart, not suitable for distributed systems

```python
# This is the default - no configuration needed
agent = Agent(
    system_prompt="You are a helpful assistant.",
    model=bedrock_model
)
```

#### 2. File-Based Session Manager
- **Use Case**: Single-server deployments, development with persistence
- **Pros**: Simple, survives restarts, easy to debug
- **Cons**: Not suitable for distributed systems, file I/O overhead

```python
from strands.session import FileSessionManager

session_manager = FileSessionManager(base_dir="./chat_sessions")

agent = Agent(
    system_prompt="You are a helpful assistant.",
    model=bedrock_model,
    session_manager=session_manager,
    session_id="user-123"  # Unique ID per user/conversation
)
```

#### 3. S3 Session Manager
- **Use Case**: Distributed systems, serverless applications
- **Pros**: Scalable, durable, works across multiple servers
- **Cons**: Network latency, requires AWS permissions

```python
from strands.session import S3SessionManager

session_manager = S3SessionManager(
    bucket_name="my-chatbot-sessions",
    prefix="conversations/"
)

agent = Agent(
    system_prompt="You are a helpful assistant.",
    model=bedrock_model,
    session_manager=session_manager,
    session_id="user-123"
)
```

#### 4. Custom Session Manager
- **Use Case**: High-performance distributed systems, custom requirements
- **Pros**: Optimized for your specific use case, low latency
- **Cons**: Requires custom implementation

You can implement your own session manager by extending the `SessionManager` interface. This allows you to use any storage backend (DynamoDB, Redis, PostgreSQL, etc.):

```python
from strands.session import SessionManager

class CustomSessionManager(SessionManager):
    def __init__(self, connection_params):
        # Initialize your storage backend
        pass
    
    def save_session(self, session_id, messages):
        # Save messages to your storage
        pass
    
    def load_session(self, session_id):
        # Load messages from your storage
        pass

session_manager = CustomSessionManager(connection_params={...})
agent = Agent(
    system_prompt="You are a helpful assistant.",
    model=bedrock_model,
    session_manager=session_manager,
    session_id="user-123"
)
```

### Choosing the Right Session Manager

| Scenario | Recommended Solution |
|----------|---------------------|
| Quick prototyping | In-Memory (default) |
| Single-server deployment | FileSessionManager |
| Serverless (Lambda) | S3SessionManager |
| High-traffic distributed | Custom (DynamoDB, Redis) |
| Multi-region | Custom with replication |

### Session Management with Guardrails

When combining session management with guardrails, the redaction features become especially important:

- **Without Redaction**: Blocked content is stored in session history, potentially causing issues on reload
- **With Redaction**: Clean conversation history that's safe to persist and reload

This is why we recommend enabling `guardrail_redact_input` and `guardrail_redact_output` for production chatbots with session persistence.

**Note:** The `redacting_chatbot` we created above is already production-ready! It combines all these concepts and is ready to be deployed with your choice of session manager.

### Additional Challenge
Create a guardrail for a corporate HR chatbot that must comply with employment law and protect sensitive workplace information.

Prerequisites:

• Access to AWS Bedrock Console

• Navigate to Bedrock > Guardrails section

• Click "Create guardrail"

Tasks:

1. Content Policy Setup: Configure content filters to prevent workplace harassment while allowing professional discussions. Consider what content
types are most critical in an HR environment.
  
    For example: hate, insults.

2. Denied Topics Configuration: Create topic policies that prevent the chatbot from handling sensitive HR matters that require human 
intervention. Think about what HR discussions could create legal liability.
  
    For example: Salary negotiations and compensation.

3. Word Filtering Strategy: Identify and block specific terms related to sensitive HR processes. Consider both legal terminology and workplace 
conflict language.
  
    For example: "confidential salary", "labor violation"

4. PII Protection Design: Determine which employee identifiers should be blocked vs anonymized. Consider what information HR systems typically 
handle.
  
    For example: Email, phone number, bank account number

5. Blocked Message Crafting: Write professional, helpful blocked messages that redirect users to appropriate HR channels without revealing why 
content was blocked.

---

# Summary

Congratulations! In this notebook, you learned how to build secure, production-ready chatbots by combining:

## 1. Amazon Bedrock Guardrails
- **Content Filters**: Block harmful content with configurable strength levels
- **Denied Topics**: Prevent specific business topics (e.g., fiduciary advice)
- **Word Filters**: Block specific words/phrases and profanity
- **PII Protection**: Detect and handle sensitive information (BLOCK or ANONYMIZE)
- **Contextual Grounding**: Prevent hallucinations with factual accuracy checks
- **Version Management**: DRAFT for testing, numbered versions for production

## 2. Strands Agent Integration
- **Simple Configuration**: Add guardrails via BedrockModel parameters
- **Automatic Protection**: Every interaction is filtered automatically
- **Redaction Options**: Control how blocked content appears in history
- **Trace Support**: Enable debugging with `guardrail_trace="enabled"`

## 3. Conversation Management
- **Built-in Memory**: Automatic conversation context management
- **Session Persistence**: FileSessionManager, S3SessionManager, or custom
- **Production Ready**: Scalable solutions for distributed systems
- **Clean History**: Redaction ensures safe conversation persistence

## Best Practices for Production

1. **Guardrail Development**:
   - Start with DRAFT versions for testing
   - Test thoroughly with real-world scenarios
   - Create numbered versions for production
   - Use meaningful version descriptions

2. **Integration**:
   - Enable `guardrail_trace` during development
   - Use `guardrail_redact_input` and `guardrail_redact_output` for chatbots
   - Provide clear custom redaction messages
   - Monitor guardrail interventions

3. **Session Management**:
   - Choose the right session manager for your architecture
   - Use unique session IDs per user/conversation
   - Implement session cleanup/expiration policies
   - Consider data residency requirements

4. **Monitoring and Iteration**:
   - Track guardrail intervention rates
   - Collect user feedback on blocked content
   - Iterate on guardrail configurations
   - Test new versions before promoting to production

## Next Steps

- Explore custom tools and function calling with secure agents
- Implement multi-agent architectures with shared guardrails
- Build RAG applications with contextual grounding checks
- Deploy your secure chatbot to production (Lambda, ECS, etc.)

## Additional Resources

- [Amazon Bedrock Guardrails Documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html)
- [Strands SDK Documentation](https://github.com/awslabs/strands)
- [AWS Well-Architected Framework - AI/ML Lens](https://docs.aws.amazon.com/wellarchitected/latest/machine-learning-lens/welcome.html)